In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import airo_models
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from tools.visualization import add_meshcat_triad
from tools.scenes import *
from tools.building import *
from typing import Optional, Tuple

from pydrake.geometry import Meshcat, MeshcatVisualizer, MeshcatVisualizerParams, Role
from pydrake.planning import RobotDiagram, RobotDiagramBuilder
from pydrake.systems.framework import Context
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

In [2]:
#
# Setup initial robot diagram
#

robot_diagram_builder = RobotDiagramBuilder()
scene_graph = robot_diagram_builder.scene_graph()
builder = robot_diagram_builder.builder()

# Adding Meshcat must also be done before finalizing
meshcat = Meshcat()
meshcat.SetCameraPose([-2.0, 0, 1.0], [0, 0, 0])
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Add visualizer for proximity/collision geometry
collision_params = MeshcatVisualizerParams(role=Role.kProximity, prefix="collision", visible_by_default=False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph.get_query_output_port(), meshcat, collision_params)

# Add robot to builder
arm_index = add_ur3e_and_table_to_builder(robot_diagram_builder)
tool_index = add_probe_tool_to_builder(robot_diagram_builder, arm_index)

# Finalize build
diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index, tool_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)


INFO:drake:Meshcat listening for connections at http://localhost:7000
Failed to load material file(s). Use default material.
material [ 'probeMaterial0' ] not found in .mtl

INFO:drake:Allocating contexts to support implicit context parallelism 12


In [3]:
#
# Setup initial pose and planner
#

start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

add_meshcat_triad(meshcat, "World", X_W_Triad=RigidTransform(p=[0, 0, 0], rpy=RollPitchYaw([0, 0, 0])))

# Planner setup
from ur_analytic_ik import ur3e, ur5e
from tools.ompl.single_arm_planner import SingleArmOmplPlanner
from tools.visualization import publish_joint_path

tcp_transform = np.identity(4)
tcp_transform[2, 3] += 0.03

def inverse_kinematics_fn(tcp_pose):
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(np.ascontiguousarray(tcp_pose), tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions

planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree, inverse_kinematics_fn)

In [4]:
#
# Class used to interface with robot
# 

from tools.interfaces.drake_robot import DrakeRobot
            
robot = DrakeRobot(diagram, context, arm_index, meshcat)
robot.zeroTFSensor()

print(robot.getTFValue())

[0. 0. 0. 0. 0. 0.]


In [5]:
#
# Controller
#

from tools.controllers.calibration import CalibrationController
from tools.visualization import *

controller = CalibrationController(robot, planner)

joint_t, time_t, transform = controller.translate_probe([0.05, 0, 0])
print(transform)

publish_ik_solutions(planner._ik_solutions, 3, meshcat, diagram, context, arm_index)

add_meshcat_triad(meshcat, "TCP Frame", X_W_Triad=RigidTransform(transform))

2024-02-29 12:02:40.970 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:156 - IK returned 8 solutions.
2024-02-29 12:02:40.971 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:167 - Found 8/8 solutions within joint bounds.
2024-02-29 12:02:40.974 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:174 - Found 6/8 valid solutions.


[[-6.1232340e-17 -1.0000000e+00  2.4492936e-16  2.9860000e-01]
 [-1.0000000e+00  6.1232340e-17 -1.2246468e-16 -1.1235000e-01]
 [ 1.2246468e-16 -2.4492936e-16 -1.0000000e+00  3.1365000e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
Debug:   RRTConnect: Planner range detected to be 6.156239
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.027601 seconds
Info:    SimpleSetup: Path simplification took 0.115290 seconds and changed from 3 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (3 start + 2 goal)
Info:    Solution found in 0.053877 seconds
Info:    SimpleSetup: Path simplification took 0.172067 seconds and changed from 4 to 2 states
Info:    SimpleSetup: Path simplification too

2024-02-29 12:02:47.188 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:203 - Found 5 paths towards IK solutions:
2024-02-29 12:02:47.189 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:204 - Path lengths: [5.56, 5.552, 4.894, 0.781, 5.629]
2024-02-29 12:02:47.189 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:205 - Path distances: [5.56, 5.552, 4.894, 0.781, 5.629]
2024-02-29 12:02:47.190 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:220 - Length of chosen solution (= shortest path): 0.781


Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 173 states (83 start + 90 goal)
Info:    ProblemDefinition: Adding approximate solution from planner RRTConnect
Info:    Solution found in 5.016751 seconds
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.037437 seconds
Info:    SimpleSetup: Path simplification took 0.099314 seconds and changed from 3 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (3 start + 2 goal)
Info:    Solution found in 0.043534 seconds
Info:    SimpleSetup: Path simplification took 0.159251 seconds and changed from 4 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states

In [6]:
robot.publish_trajectory(joint_t, time_t)
print(robot.getTCPPose()[1])

[[-1.24900090e-16 -1.00000000e+00 -1.10089842e-15  2.19159629e-01]
 [-1.00000000e+00  1.31838984e-16 -1.85772707e-16 -9.92317674e-02]
 [ 1.85903902e-16  1.03962435e-15 -1.00000000e+00  2.34615707e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
